In [1]:
import os
import json

### Read in data

In [2]:
folder = 'data/ria/'
files = os.listdir(folder)

In [3]:
oracle_summary_errors = 0
all_train_data = []
train_files = [file for file in files if 'ria_train' in file]
for file in train_files:
    for line in open(folder + file, 'rb'):
        obj = json.loads(line)
        if obj['oracle_sentences'] == list():
            oracle_summary_errors = oracle_summary_errors + 1
        else:
            all_train_data.append(obj)
print("Total train samples: ", len(all_train_data))
print("Total errors in train: ", oracle_summary_errors)

oracle_summary_errors = 0
all_test_data = []
test_files = [file for file in files if 'ria_test' in file]
for file in test_files:
    for line in open(folder + file, 'rb'):
        obj = json.loads(line)
        if obj['oracle_sentences'] == list():
            oracle_summary_errors = oracle_summary_errors + 1
        else:
            all_test_data.append(obj)
print("Total test samples: ", len(all_test_data))
print("Total errors in train: ", oracle_summary_errors)

Total train samples:  971144
Total errors in train:  207
Total test samples:  19825
Total errors in train:  4


In [4]:
all_test_data[0]

{'text': 'Большая часть из 33 детей, которых граждане сша пытались вывезти из гаити в организованный в доминиканской республике приют, не являются сиротами, сообщает в воскресенье агентство франс пресс со ссылкой на заявление представителя международной организации "детские деревни sos" (sos children\'s village), оказывающей помощь детям, оставшимся без родителей Как заявила агентству патрисия варгас (patricia vargas), курирующая программы "детских деревень sos" в центральной америке, мексике и на карибах, поговорив с детьми она выяснила, что родители многих из них живы. Некоторые дети смогли назвать свои домашние адреса и номера телефонов, что дает возможность связаться с их родителями. В это воскресенье гаитянская полиция задержала десятерых граждан сша, подозреваемых в попытке без разрешения вывезти более 30 детей в доминиканскую республику. Представитель баптистской церкви в городе меридиан американского штата айдахо шон лэнкфорд (sean lankford) заявил, что задержанные прибыли на г

In [5]:
import pickle

# with open('all_train_data.pkl', 'wb') as f:
#     pickle.dump(all_train_data, f)
# with open('all_test_data.pkl', 'wb') as f:
#     pickle.dump(all_test_data, f)

### Baseline approach

BPE

In [23]:
from sentencepiece import SentencePieceTrainer

def train_bpe(records, model_path, model_type="bpe", vocab_size=15000, lower=True):
    temp_file_name = "temp.txt"
    with open(temp_file_name, "wb") as temp:
        for record in records:
            summary = record["title"].strip()
            text = record["text"].strip()
            if lower:
                summary = summary.lower()
                text = text.lower()
            if not text or not summary:
                continue
            temp.write((text + "\n").encode('utf8'))
            temp.write((summary + "\n").encode('utf8'))
    if not os.path.exists(model_path):
        os.makedirs(model_path)
    cmd = "--input={} --model_prefix={} --vocab_size={} --model_type={}".format(
        temp_file_name,
        os.path.join(model_path, model_type),
        vocab_size,
        model_type)
    SentencePieceTrainer.Train(cmd)


train_bpe(all_train_data, "./")

In [24]:
!head bpe.vocab
!cat bpe.vocab | wc -l

<unk>	0
<s>	0
</s>	0
�����	-0
�����	-1
�����	-2
����	-3
����	-4
����	-5
����	-6
15000


In [6]:
from sentencepiece import SentencePieceProcessor

def bpe_tokenize(text, bpe_processor):
    return bpe_processor.EncodeAsPieces(text)

In [25]:
bpe_processor = SentencePieceProcessor()
bpe_processor.Load("bpe.model")
bpe_tokenize("октябрь богат на изменения", bpe_processor)

['▁октябрь', '▁бога', 'т', '▁на', '▁изменения']

Vocabulary

In [7]:
from collections import Counter
from typing import List, Tuple
import os

class Vocabulary:
    def __init__(self):
        self.index2word = list()
        self.word2index = dict()
        self.word2count = Counter()
        self.reset()

    def get_pad(self):
        return self.word2index["<pad>"]

    def get_sos(self):
        return self.word2index["<sos>"]

    def get_eos(self):
        return self.word2index["<eos>"]

    def get_unk(self):
        return self.word2index["<unk>"]

    def add_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = len(self.index2word)
            self.word2count[word] += 1
            self.index2word.append(word)
        else:
            self.word2count[word] += 1
    
    def has_word(self, word) -> bool:
        return word in self.word2index

    def get_index(self, word):
        if word in self.word2index:
            return self.word2index[word]
        return self.get_unk()

    def get_word(self, index):
        return self.index2word[index]

    def size(self):
        return len(self.index2word)

    def is_empty(self):
        empty_size = 4
        return self.size() <= empty_size

    def shrink(self, n):
        best_words = self.word2count.most_common(n)
        self.reset()
        for word, count in best_words:
            self.add_word(word)
            self.word2count[word] = count

    def reset(self):
        self.word2count = Counter()
        self.index2word = ["<pad>", "<sos>", "<eos>", "<unk>"]
        self.word2index = {word: index for index, word in enumerate(self.index2word)}

In [27]:
def build_vocabulary(records, bpe_processor, lower=True): 
    vocabulary = Vocabulary()
    for record in records:
        text = record["text"]
        text = text.lower() if lower else text
        tokens = bpe_tokenize(text, bpe_processor)
        for token in tokens:
            vocabulary.add_word(token)
    return vocabulary

vocabulary = build_vocabulary(all_train_data, bpe_processor)
print(vocabulary.word2count.most_common(100)) 

[(',', 18168256), ('.', 12392015), ('▁в', 12005340), ('▁и', 6432972), ('▁на', 5083617), ('▁"', 4875722), ('▁по', 3275392), ('-', 2961503), ('▁с', 2843450), ('▁-', 2309866), ('▁что', 2242613), ('▁не', 2242492), ('"', 2035120), ('",', 1895808), ('▁—', 1601830), ('▁из', 1334102), ('▁за', 1267384), ('▁(', 1232091), ('▁о', 1196322), ('▁а', 1104326), ('▁к', 1071177), ('▁от', 932615), ('▁для', 927934), ('▁года', 917722), ('▁до', 901930), ('▁у', 825459), ('▁', 812073), ('▁как', 808692), ('".', 799409), (')', 789735), ('ли', 783181), ('л', 773223), ('▁при', 772246), ('на', 740021), ('▁он', 722335), ('м', 708348), ('ла', 702955), ('▁его', 701044), ('но', 692294), ('т', 680721), ('▁россии', 659036), ('ной', 646428), ('ка', 644309), (':', 638636), ('▁это', 633273), ('в', 628126), ('▁также', 622859), ('ть', 610929), (').', 606723), ('ных', 606187), ('ми', 600581), ('х', 597765), ('ного', 593355), ('▁но', 580468), ('▁во', 567119), ('та', 562942), ('я', 555759), ('с', 555366), ('▁будет', 551564), ('н

Batch Generator

In [8]:
import pickle
# with open('bpe_processor.pkl', 'wb') as f:
#     pickle.dump(bpe_processor, f)
# with open('vocabulary.pkl', 'wb') as f:
#     pickle.dump(vocabulary, f)
with open('bpe_processor.pkl', 'rb') as f:
    bpe_processor = pickle.load(f)
with open('vocabulary.pkl', 'rb') as f:
    vocabulary = pickle.load(f)

In [9]:
import random
import math
import razdel
import torch
import numpy as np
from rouge import Rouge


class BatchIterator():
    def __init__(self, records, vocabulary, batch_size, bpe_processor, shuffle=True, lower=True, max_sentences=30, max_sentence_length=50, device=torch.device('cpu')):
        self.records = records
        self.num_samples = len(records)
        self.batch_size = batch_size
        self.bpe_processor = bpe_processor
        self.shuffle = shuffle
        self.batches_count = int(math.ceil(self.num_samples / batch_size))
        self.lower = lower
        self.rouge = Rouge()
        self.vocabulary = vocabulary
        self.max_sentences = max_sentences
        self.max_sentence_length = max_sentence_length
        self.device = device
        
    def __len__(self):
        return self.batches_count
    
    def __iter__(self):
        indices = np.arange(self.num_samples)
        if self.shuffle:
            np.random.shuffle(indices)

        for start in range(0, self.num_samples, self.batch_size):
            end = min(start + self.batch_size, self.num_samples)
            batch_indices = indices[start:end]
            batch_inputs = []
            batch_outputs = []
            max_sentence_length = 0
            max_sentences = 0
            batch_records = []
            # свой max_sentences для батча
            for data_ind in batch_indices:
                record = self.records[data_ind]
                batch_records.append(record)
                text = record["text"]
                summary = record["title"]
                summary = summary.lower() if self.lower else summary

                if "sentences" not in record:
                    sentences = [sentence.text.lower() if self.lower else sentence.text for sentence in razdel.sentenize(text)][:self.max_sentences]
                else:
                    sentences = record["sentences"]
                max_sentences = max(len(sentences), max_sentences)

                if "oracle_sentences" not in record:
                    calc_score = lambda x, y: calc_single_score(x, y, self.rouge)
                    sentences_indicies = build_oracle_summary_greedy(text, summary, calc_score=calc_score, lower=self.lower, max_sentences=self.max_sentences)[1]
                else:
                    sentences_indicies = record["oracle_sentences"]
                # len(sentences)
                inputs = [list(map(self.vocabulary.get_index, bpe_tokenize(sentence, self.bpe_processor)[:self.max_sentence_length])) for sentence in sentences]
                max_sentence_length = max(max_sentence_length, max([len(tokens) for tokens in inputs]))
                # len(sentences)
                outputs = [int(i in sentences_indicies) for i in range(len(sentences))]
                batch_inputs.append(inputs)
                batch_outputs.append(outputs)
            tensor_inputs = torch.zeros((self.batch_size, max_sentences, max_sentence_length), dtype=torch.long, device=self.device)
            tensor_outputs = torch.zeros((self.batch_size, max_sentences), dtype=torch.float32, device=self.device)
            for i, inputs in enumerate(batch_inputs):
                for j, sentence_tokens in enumerate(inputs):
                    tensor_inputs[i][j][:len(sentence_tokens)] = torch.LongTensor(sentence_tokens)
            for i, outputs in enumerate(batch_outputs):
                tensor_outputs[i][:len(outputs)] = torch.LongTensor(outputs)

            yield {
                'inputs': tensor_inputs,
                'outputs': tensor_outputs,
                'records': batch_records
            }

In [10]:
train_iterator = BatchIterator(all_train_data, vocabulary, 4, bpe_processor)

In [11]:
for batch in train_iterator:
    print(batch)
    break

{'inputs': tensor([[[   27,     3,   208,  ...,   441,  9717,   181],
         [   27,     3,  6237,  ...,     0,     0,     0],
         [   27,     3, 11353,  ...,     0,     0,     0],
         ...,
         [   27,     3,    23,  ...,     0,     0,     0],
         [   27,     3,  1535,  ...,     0,     0,     0],
         [   27,     3,  1384,  ...,     0,     0,     0]],

        [[   27,     3,   208,  ...,     0,     0,     0],
         [   27,     3,   208,  ...,     0,     0,     0],
         [  109,   820,  1231,  ...,  3994,  7337,   546],
         ...,
         [    0,     0,     0,  ...,     0,     0,     0],
         [    0,     0,     0,  ...,     0,     0,     0],
         [    0,     0,     0,  ...,     0,     0,     0]],

        [[   27,     3,  1165,  ...,     0,     0,     0],
         [    0,     0,     0,  ...,     0,     0,     0],
         [    0,     0,     0,  ...,     0,     0,     0],
         ...,
         [    0,     0,     0,  ...,     0,     0,     0],

In [12]:
batch['inputs'].shape

torch.Size([4, 13, 50])

In [13]:
batch['outputs'].shape

torch.Size([4, 13])

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
import time

def train_model(model, train_records, val_records, vocabulary, bpe_processor, batch_size=32,
                epochs_count=10, loss_every_nsteps=16, lr=0.001, device_name="cuda"):
    params_count = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print("Trainable params: {}".format(params_count))
    device = torch.device(device_name)
    model = model.to(device)
    total_loss = 0
    start_time = time.time()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    loss_function = nn.BCEWithLogitsLoss().to(device)
    for epoch in range(epochs_count):
        for step, batch in enumerate(BatchIterator(train_records, vocabulary, batch_size, bpe_processor, device=device)):
            model.train()
            logits = model(batch["inputs"]) # Прямой проход
            loss = loss_function(logits, batch["outputs"]) # Подсчёт ошибки
            loss.backward() # Подсчёт градиентов dL/dw
            optimizer.step() # Градиентный спуск или его модификации (в данном случае Adam)
            optimizer.zero_grad() # Зануление градиентов, чтобы их спокойно менять на следующей итерации
            total_loss += loss.item()
            if step % loss_every_nsteps == 0 and step != 0:
                val_total_loss = 0
                val_batch_count = 0
                model.eval()
#                 for _, val_batch in enumerate(BatchIterator(val_records, vocabulary, batch_size, bpe_processor, device=device)):
#                     logits = model(val_batch["inputs"]) # Прямой проход
#                     val_total_loss += loss_function(logits, val_batch["outputs"]) # Подсчёт ошибки
#                     val_batch_count += 1
#                 avg_val_loss = val_total_loss/val_batch_count
#                 print("Epoch = {}, Avg Train Loss = {:.4f}, Avg val loss = {:.4f}, Time = {:.2f}s".format(epoch, total_loss / loss_every_nsteps, avg_val_loss, time.time() - start_time))
                print("Epoch = {}, Avg Train Loss = {:.4f}, Time = {:.2f}s".format(epoch, total_loss / loss_every_nsteps, time.time() - start_time))
                total_loss = 0
                start_time = time.time()
        total_loss = 0
        start_time = time.time()

In [11]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

from torch.nn.utils.rnn import pack_padded_sequence as pack
from torch.nn.utils.rnn import pad_packed_sequence as unpack

class SentenceEncoderRNN(nn.Module):
    def __init__(self, input_size, embedding_dim, hidden_size, n_layers=3, dropout=0.3, bidirectional=True):
        super(SentenceEncoderRNN, self).__init__()

        num_directions = 2 if bidirectional else 1
        assert hidden_size % num_directions == 0
        hidden_size = hidden_size // num_directions

        self.embedding_dim = embedding_dim
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.dropout = dropout
        self.bidirectional = bidirectional

        self.embedding_layer = nn.Embedding(input_size, embedding_dim)
        self.rnn_layer = nn.LSTM(embedding_dim, hidden_size, n_layers, dropout=dropout, bidirectional=bidirectional, batch_first=True)
        self.dropout_layer = nn.Dropout(dropout)

    def forward(self, inputs, hidden=None):
        embedded = self.embedding_layer(inputs)
        outputs, _ = self.rnn_layer(embedded, hidden)
        sentences_embeddings = torch.mean(outputs, 1)
        return sentences_embeddings

class SentenceTaggerRNN(nn.Module):
    def __init__(self,
                 vocabulary_size,
                 token_embedding_dim=256,
                 sentence_encoder_hidden_size=256,
                 hidden_size=256,
                 bidirectional=True,
                 sentence_encoder_n_layers=2,
                 sentence_encoder_dropout=0.3,
                 sentence_encoder_bidirectional=True,
                 n_layers=1,
                 dropout=0.3):
        super(SentenceTaggerRNN, self).__init__()

        num_directions = 2 if bidirectional else 1
        assert hidden_size % num_directions == 0
        hidden_size = hidden_size // num_directions

        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.dropout = dropout
        self.bidirectional = bidirectional

        self.sentence_encoder = SentenceEncoderRNN(vocabulary_size, token_embedding_dim,
                                                   sentence_encoder_hidden_size, sentence_encoder_n_layers, 
                                                   sentence_encoder_dropout, sentence_encoder_bidirectional)
        self.rnn_layer = nn.LSTM(sentence_encoder_hidden_size, hidden_size, n_layers, dropout=dropout,
                           bidirectional=bidirectional, batch_first=True)
        self.dropout_layer = nn.Dropout(dropout)
        self.content_linear_layer = nn.Linear(hidden_size * 2, 1)
        self.document_linear_layer = nn.Linear(hidden_size * 2, hidden_size * 2)
        self.salience_linear_layer = nn.Linear(hidden_size * 2, hidden_size * 2)
        self.tanh_layer = nn.Tanh()

    def forward(self, inputs, hidden=None):
        batch_size = inputs.size(0)
        sentences_count = inputs.size(1)
        tokens_count = inputs.size(2)
        inputs = inputs.reshape(-1, tokens_count)
        embedded_sentences = self.sentence_encoder(inputs)
        embedded_sentences = embedded_sentences.reshape(batch_size, sentences_count, -1)
        outputs, _ = self.rnn_layer(embedded_sentences, hidden)
        outputs = self.dropout_layer(outputs)
        document_embedding = self.tanh_layer(self.document_linear_layer(torch.mean(outputs, 1)))
        content = self.content_linear_layer(outputs).squeeze(2)
        salience = torch.bmm(outputs, self.salience_linear_layer(document_embedding).unsqueeze(2)).squeeze(2)
        return content + salience

In [19]:
model = SentenceTaggerRNN(vocabulary.size())
train_model(model, all_train_data, all_test_data, vocabulary, bpe_processor, device_name="cuda", batch_size=32)

Trainable params: 5356289
Epoch = 0, Avg Train Loss = 0.1899, Time = 4.49s
Epoch = 0, Avg Train Loss = 0.0900, Time = 4.08s
Epoch = 0, Avg Train Loss = 0.0862, Time = 4.16s
Epoch = 0, Avg Train Loss = 0.0852, Time = 4.09s
Epoch = 0, Avg Train Loss = 0.0784, Time = 3.99s
Epoch = 0, Avg Train Loss = 0.0751, Time = 4.08s
Epoch = 0, Avg Train Loss = 0.0758, Time = 4.25s
Epoch = 0, Avg Train Loss = 0.0749, Time = 4.09s
Epoch = 0, Avg Train Loss = 0.0702, Time = 4.12s
Epoch = 0, Avg Train Loss = 0.0667, Time = 4.10s
Epoch = 0, Avg Train Loss = 0.0739, Time = 3.96s
Epoch = 0, Avg Train Loss = 0.0679, Time = 4.07s
Epoch = 0, Avg Train Loss = 0.0678, Time = 4.02s
Epoch = 0, Avg Train Loss = 0.0658, Time = 4.07s
Epoch = 0, Avg Train Loss = 0.0662, Time = 4.19s
Epoch = 0, Avg Train Loss = 0.0701, Time = 4.11s
Epoch = 0, Avg Train Loss = 0.0694, Time = 4.08s
Epoch = 0, Avg Train Loss = 0.0733, Time = 4.06s
Epoch = 0, Avg Train Loss = 0.0662, Time = 4.16s
Epoch = 0, Avg Train Loss = 0.0674, Time = 

KeyboardInterrupt: 

1 эпоха - примерно 16 часов на моем компьютере... Проверим так, как есть сейчас

Оценка качества

In [23]:
from nltk.translate.bleu_score import corpus_bleu
from rouge import Rouge

def calc_scores(references, predictions, metric="all"):
    print("Count:", len(predictions))
    print("Ref:", references[-1])
    print("Hyp:", predictions[-1])

    if metric in ("bleu", "all"):
        print("BLEU: ", corpus_bleu([[r] for r in references], predictions))
    if metric in ("rouge", "all"):
        rouge = Rouge()
        scores = rouge.get_scores(predictions, references, avg=True)
        print("ROUGE: ", scores)

In [24]:
device = torch.device("cuda")

references = []
predictions = []
for step, batch in enumerate(BatchIterator(all_test_data[:1000], vocabulary, 32, bpe_processor, device=device)):
    logits = model(batch["inputs"]) # Прямой проход
    records = batch["records"]
    for record, record_logits in zip(records, logits):
        sentences = record["sentences"]
        predicted_summary = []
        for i, logit in enumerate(record_logits):
            if logit > 0.0:
                predicted_summary.append(sentences[i])
        if not predicted_summary:
            predicted_summary.append(sentences[torch.max(record_logits, dim=0)[1].item()])
        predicted_summary = " ".join(predicted_summary)
        references.append(record["title"].lower())
        predictions.append(predicted_summary)

calc_scores(references, predictions)

Count: 1000
Ref: заявления жителей "речника" о вывозе вещей рассмотрят по закону - фссп
Hyp: Заявления жителей поселка "речник" на западе москвы о предоставлении времени для вывоза имущества и строений будут рассматриваться в установленном законом порядке, сообщил в понедельник заместитель руководителя столичного управления федеральной службы судебных приставов евгений лукьянчиков.
BLEU:  0.2023709767449148
ROUGE:  {'rouge-1': {'f': 0.20992510106228962, 'p': 0.14500262979745654, 'r': 0.4194260128760132}, 'rouge-2': {'f': 0.09131963287817384, 'p': 0.06231760421922613, 'r': 0.18984801309801333}, 'rouge-l': {'f': 0.1988034844595074, 'p': 0.13879161888887406, 'r': 0.38346827339327355}}


### Advanced approach

+ +positional embeddings
+ +bias

In [80]:
import random
import math
import razdel
import torch
import numpy as np
from rouge import Rouge


class BatchIterator():
    def __init__(self, records, vocabulary, batch_size, bpe_processor, shuffle=True, lower=True, max_sentences=30, max_sentence_length=50, device=torch.device('cpu')):
        self.records = records
        self.num_samples = len(records)
        self.batch_size = batch_size
        self.bpe_processor = bpe_processor
        self.shuffle = shuffle
        self.batches_count = int(math.ceil(self.num_samples / batch_size))
        self.lower = lower
        self.rouge = Rouge()
        self.vocabulary = vocabulary
        self.max_sentences = max_sentences
        self.max_sentence_length = max_sentence_length
        self.device = device
        
    def __len__(self):
        return self.batches_count
    
    def __iter__(self):
        indices = np.arange(self.num_samples)
        if self.shuffle:
            np.random.shuffle(indices)

        for start in range(0, self.num_samples, self.batch_size):
            end = min(start + self.batch_size, self.num_samples)
            batch_indices = indices[start:end]
            batch_inputs = []
            batch_outputs = []
            rel_pos_inputs = []
            max_sentence_length = 0
            max_sentences = 0
            batch_records = []
            # свой max_sentences для батча
            for data_ind in batch_indices:
                record = self.records[data_ind]
                batch_records.append(record)
                text = record["text"]
                summary = record["title"]
                summary = summary.lower() if self.lower else summary

                if "sentences" not in record:
                    sentences = [sentence.text.lower() if self.lower else sentence.text for sentence in razdel.sentenize(text)][:self.max_sentences]
                else:
                    sentences = record["sentences"]
                max_sentences = max(len(sentences), max_sentences)

                if "oracle_sentences" not in record:
                    calc_score = lambda x, y: calc_single_score(x, y, self.rouge)
                    sentences_indicies = build_oracle_summary_greedy(text, summary, calc_score=calc_score, lower=self.lower, max_sentences=self.max_sentences)[1]
                else:
                    sentences_indicies = record["oracle_sentences"]
                # len(sentences)
                inputs = [list(map(self.vocabulary.get_index, bpe_tokenize(sentence, self.bpe_processor)[:self.max_sentence_length])) for sentence in sentences]
                max_sentence_length = max(max_sentence_length, max([len(tokens) for tokens in inputs]))
                # len(sentences)
                outputs = [int(i in sentences_indicies) for i in range(len(sentences))]
                # rel_pos
                pos = [i / len(sentences) for i in range(len(sentences))]
                bins = np.array([0.25, 0.5, 0.75, 1.0])
                # possible options array([1, 2, 3, 4, 5])
                rel_pos_inds = np.digitize(np.array([p for p in pos]), bins) + 1
                rel_pos_inputs.append(rel_pos_inds)
                batch_inputs.append(inputs)
                batch_outputs.append(outputs)
            tensor_inputs = torch.zeros((self.batch_size, max_sentences, max_sentence_length), dtype=torch.long, device=self.device)
            tensor_outputs = torch.zeros((self.batch_size, max_sentences), dtype=torch.float32, device=self.device)
            tensor_abs_pos = torch.zeros((self.batch_size, max_sentences), dtype=torch.long, device=self.device)
            tensor_rel_pos = torch.zeros((self.batch_size, max_sentences), dtype=torch.long, device=self.device)
            for i, inputs in enumerate(batch_inputs):
                for j, sentence_tokens in enumerate(inputs):
                    tensor_inputs[i][j][:len(sentence_tokens)] = torch.LongTensor(sentence_tokens)
            for i, outputs in enumerate(batch_outputs):
                tensor_outputs[i][:len(outputs)] = torch.LongTensor(outputs)
                # abs pos
                tensor_abs_pos[i] = torch.LongTensor([i for i in range(max_sentences)])
            # rel pos
            for i, rel_pos_input in enumerate(rel_pos_inputs):
                tensor_rel_pos[i][:len(rel_pos_input)] = torch.LongTensor(rel_pos_input)            
            
            yield {
                'inputs': (tensor_inputs, tensor_abs_pos, tensor_rel_pos),
                'outputs': tensor_outputs,
                'records': batch_records
            }

In [81]:
train_iterator = BatchIterator(all_train_data, vocabulary, 4, bpe_processor)

In [82]:
for batch in train_iterator:
    print(batch)
    break

{'inputs': (tensor([[[   27,     3,  1384,  ..., 12153,  6919,     8],
         [   27,     3,    35,  ...,     0,     0,     0],
         [   27,     3,    69,  ...,     0,     0,     0],
         ...,
         [   27,     3,  5443,  ...,     0,     0,     0],
         [   27,     3,  1384,  ...,     0,     0,     0],
         [   27,     3,  1147,  ...,     0,     0,     0]],

        [[   27,     3,  1851,  ...,     0,     0,     0],
         [   27,     3,  6347,  ...,     0,     0,     0],
         [   27,     3,  1384,  ...,     0,     0,     0],
         ...,
         [    0,     0,     0,  ...,     0,     0,     0],
         [    0,     0,     0,  ...,     0,     0,     0],
         [    0,     0,     0,  ...,     0,     0,     0]],

        [[   27,     3,  6022,  ...,     0,     0,     0],
         [   27,     3,  1147,  ...,  8186,   650,    41],
         [   27,     3,  1384,  ...,     0,     0,     0],
         ...,
         [    0,     0,     0,  ...,     0,     0,     0]

In [83]:
batch['inputs'][0].shape

torch.Size([4, 25, 50])

In [84]:
batch['inputs'][1]

tensor([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19, 20, 21, 22, 23, 24],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19, 20, 21, 22, 23, 24],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19, 20, 21, 22, 23, 24],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19, 20, 21, 22, 23, 24]])

In [85]:
batch['inputs'][2]

tensor([[1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4,
         4],
        [1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0],
        [1, 1, 2, 2, 3, 3, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0],
        [1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0]])

In [86]:
batch['outputs'].shape

torch.Size([4, 25])

In [87]:
import torch
import torch.nn as nn
import torch.optim as optim
import time

def train_model(model, train_records, val_records, vocabulary, bpe_processor, batch_size=32,
                epochs_count=10, loss_every_nsteps=16, lr=0.001, device_name="cuda"):
    params_count = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print("Trainable params: {}".format(params_count))
    device = torch.device(device_name)
    model = model.to(device)
    total_loss = 0
    start_time = time.time()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    loss_function = nn.BCEWithLogitsLoss().to(device)
    for epoch in range(epochs_count):
        for step, batch in enumerate(BatchIterator(train_records, vocabulary, batch_size, bpe_processor, device=device)):
            model.train()
            logits = model(batch["inputs"]) # Прямой проход
            loss = loss_function(logits, batch["outputs"]) # Подсчёт ошибки
            loss.backward() # Подсчёт градиентов dL/dw
            optimizer.step() # Градиентный спуск или его модификации (в данном случае Adam)
            optimizer.zero_grad() # Зануление градиентов, чтобы их спокойно менять на следующей итерации
            total_loss += loss.item()
            if step % loss_every_nsteps == 0 and step != 0:
                val_total_loss = 0
                val_batch_count = 0
                model.eval()
#                 for _, val_batch in enumerate(BatchIterator(val_records, vocabulary, batch_size, bpe_processor, device=device)):
#                     logits = model(val_batch["inputs"]) # Прямой проход
#                     val_total_loss += loss_function(logits, val_batch["outputs"]) # Подсчёт ошибки
#                     val_batch_count += 1
#                 avg_val_loss = val_total_loss/val_batch_count
#                 print("Epoch = {}, Avg Train Loss = {:.4f}, Avg val loss = {:.4f}, Time = {:.2f}s".format(epoch, total_loss / loss_every_nsteps, avg_val_loss, time.time() - start_time))
                print("Epoch = {}, Avg Train Loss = {:.4f}, Time = {:.2f}s".format(epoch, total_loss / loss_every_nsteps, time.time() - start_time))
                total_loss = 0
                start_time = time.time()
        total_loss = 0
        start_time = time.time()

In [94]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

from torch.nn.utils.rnn import pack_padded_sequence as pack
from torch.nn.utils.rnn import pad_packed_sequence as unpack

class SentenceEncoderRNN(nn.Module):
    def __init__(self, input_size, embedding_dim, hidden_size, n_layers=3, dropout=0.3, bidirectional=True):
        super(SentenceEncoderRNN, self).__init__()

        num_directions = 2 if bidirectional else 1
        assert hidden_size % num_directions == 0
        hidden_size = hidden_size // num_directions

        self.embedding_dim = embedding_dim
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.dropout = dropout
        self.bidirectional = bidirectional

        self.embedding_layer = nn.Embedding(input_size, embedding_dim)
        self.rnn_layer = nn.LSTM(embedding_dim, hidden_size, n_layers, dropout=dropout, bidirectional=bidirectional, batch_first=True)
        self.dropout_layer = nn.Dropout(dropout)

    def forward(self, inputs, hidden=None):
        embedded = self.embedding_layer(inputs)
        outputs, _ = self.rnn_layer(embedded, hidden)
        sentences_embeddings = torch.mean(outputs, 1)
        return sentences_embeddings

class SentenceTaggerRNN(nn.Module):
    def __init__(self,
                 vocabulary_size,
                 token_embedding_dim=256,
                 sentence_encoder_hidden_size=256,
                 hidden_size=256,
                 bidirectional=True,
                 sentence_encoder_n_layers=2,
                 sentence_encoder_dropout=0.3,
                 sentence_encoder_bidirectional=True,
                 n_layers=1,
                 dropout=0.3):
        super(SentenceTaggerRNN, self).__init__()

        num_directions = 2 if bidirectional else 1
        assert hidden_size % num_directions == 0
        hidden_size = hidden_size // num_directions

        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.dropout = dropout
        self.bidirectional = bidirectional

        self.sentence_encoder = SentenceEncoderRNN(vocabulary_size, token_embedding_dim,
                                                   sentence_encoder_hidden_size, sentence_encoder_n_layers, 
                                                   sentence_encoder_dropout, sentence_encoder_bidirectional)
        self.rnn_layer = nn.LSTM(sentence_encoder_hidden_size, hidden_size, n_layers, dropout=dropout,
                           bidirectional=bidirectional, batch_first=True)
        self.dropout_layer = nn.Dropout(dropout)
        self.content_linear_layer = nn.Linear(hidden_size * 2, 1)
        self.document_linear_layer = nn.Linear(hidden_size * 2, hidden_size * 2)
        self.salience_linear_layer = nn.Linear(hidden_size * 2, hidden_size * 2)
        self.tanh_layer = nn.Tanh()
        
        self.output_bias = nn.Parameter(torch.ones(1))
        # max_sentences
        self.abs_pos_emb = nn.Embedding(30, 1)
        self.rel_pos_emb = nn.Embedding(6, 1)

    def forward(self, inputs, hidden=None):
        embeddings_input = inputs[0]
        abs_pos_input = inputs[1]
        rel_pos_input = inputs[2]
        batch_size = embeddings_input.size(0)
        sentences_count = embeddings_input.size(1)
        tokens_count = embeddings_input.size(2)
        embeddings_input = embeddings_input.reshape(-1, tokens_count)
        embedded_sentences = self.sentence_encoder(embeddings_input)
        embedded_sentences = embedded_sentences.reshape(batch_size, sentences_count, -1)
        outputs, _ = self.rnn_layer(embedded_sentences, hidden)
        outputs = self.dropout_layer(outputs)
        document_embedding = self.tanh_layer(self.document_linear_layer(torch.mean(outputs, 1)))
        content = self.content_linear_layer(outputs).squeeze(2)
        salience = torch.bmm(outputs, self.salience_linear_layer(document_embedding).unsqueeze(2)).squeeze(2)
        rel_pos_embedding = self.rel_pos_emb(rel_pos_input).squeeze()
        abs_pos_embedding = self.abs_pos_emb(abs_pos_input).squeeze()
        # (batch_size, max_sentences)
        return content + salience + self.output_bias + rel_pos_embedding + abs_pos_embedding  

In [95]:
model = SentenceTaggerRNN(vocabulary.size())
train_model(model, all_train_data, all_test_data, vocabulary, bpe_processor, device_name="cuda", batch_size=32)

Trainable params: 5356326
Epoch = 0, Avg Train Loss = 0.4611, Time = 4.46s
Epoch = 0, Avg Train Loss = 0.1098, Time = 4.04s
Epoch = 0, Avg Train Loss = 0.0882, Time = 4.08s
Epoch = 0, Avg Train Loss = 0.0824, Time = 4.04s
Epoch = 0, Avg Train Loss = 0.0792, Time = 4.11s
Epoch = 0, Avg Train Loss = 0.0884, Time = 4.05s


KeyboardInterrupt: 

+ +summary relevance

In [14]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

from torch.nn.utils.rnn import pack_padded_sequence as pack
from torch.nn.utils.rnn import pad_packed_sequence as unpack

class SentenceEncoderRNN(nn.Module):
    def __init__(self, input_size, embedding_dim, hidden_size, n_layers=3, dropout=0.3, bidirectional=True):
        super(SentenceEncoderRNN, self).__init__()

        num_directions = 2 if bidirectional else 1
        assert hidden_size % num_directions == 0
        hidden_size = hidden_size // num_directions

        self.embedding_dim = embedding_dim
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.dropout = dropout
        self.bidirectional = bidirectional

        self.embedding_layer = nn.Embedding(input_size, embedding_dim)
        self.rnn_layer = nn.LSTM(embedding_dim, hidden_size, n_layers, dropout=dropout, bidirectional=bidirectional, batch_first=True)
        self.dropout_layer = nn.Dropout(dropout)

    def forward(self, inputs, hidden=None):
        embedded = self.embedding_layer(inputs)
        outputs, _ = self.rnn_layer(embedded, hidden)
        sentences_embeddings = torch.mean(outputs, 1)
        return sentences_embeddings

class SentenceTaggerRNN(nn.Module):
    def __init__(self,
                 vocabulary_size,
                 token_embedding_dim=256,
                 sentence_encoder_hidden_size=256,
                 hidden_size=256,
                 bidirectional=True,
                 sentence_encoder_n_layers=2,
                 sentence_encoder_dropout=0.3,
                 sentence_encoder_bidirectional=True,
                 n_layers=1,
                 dropout=0.3):
        super(SentenceTaggerRNN, self).__init__()

        num_directions = 2 if bidirectional else 1
        assert hidden_size % num_directions == 0
        hidden_size = hidden_size // num_directions

        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.dropout = dropout
        self.bidirectional = bidirectional

        self.sentence_encoder = SentenceEncoderRNN(vocabulary_size, token_embedding_dim,
                                                   sentence_encoder_hidden_size, sentence_encoder_n_layers, 
                                                   sentence_encoder_dropout, sentence_encoder_bidirectional)
        self.rnn_layer = nn.LSTM(sentence_encoder_hidden_size, hidden_size, n_layers, dropout=dropout,
                           bidirectional=bidirectional, batch_first=True)
        self.dropout_layer = nn.Dropout(dropout)
        self.content_linear_layer = nn.Linear(hidden_size * 2, 1)
        self.document_linear_layer = nn.Linear(hidden_size * 2, hidden_size * 2)
        self.salience_linear_layer = nn.Linear(hidden_size * 2, hidden_size * 2)
        self.tanh_layer = nn.Tanh()
        
        self.output_bias = nn.Parameter(torch.ones(1))
        self.abs_pos_emb = nn.Embedding(30, 1)
        self.rel_pos_emb = nn.Embedding(6, 1)
        self.novelty_tanh_layer = nn.Tanh()
        self.novelty_linear_layer = nn.Linear(hidden_size * 2, hidden_size * 2)

    def forward(self, inputs, hidden=None):
        embeddings_input = inputs[0]
        abs_pos_input = inputs[1]
        rel_pos_input = inputs[2]
        batch_size = embeddings_input.size(0)
        sentences_count = embeddings_input.size(1)
        tokens_count = embeddings_input.size(2)
        embeddings_input = embeddings_input.reshape(-1, tokens_count)
        embedded_sentences = self.sentence_encoder(embeddings_input)
        embedded_sentences = embedded_sentences.reshape(batch_size, sentences_count, -1)
        outputs, _ = self.rnn_layer(embedded_sentences, hidden)
        outputs = self.dropout_layer(outputs) # h [32, 30, 256]
        document_embedding = self.tanh_layer(self.document_linear_layer(torch.mean(outputs, 1))) # [32, 256]
        content = self.content_linear_layer(outputs).squeeze(2)
        # self.salience_linear_layer(document_embedding).unsqueeze(2) [32, 256, 1]
        # [32, 30, 256] * [32, 256, 1] -> [32, 30]
        salience = torch.bmm(outputs, self.salience_linear_layer(document_embedding).unsqueeze(2)).squeeze(2)
        rel_pos_embedding = self.rel_pos_emb(rel_pos_input).squeeze()
        abs_pos_embedding = self.abs_pos_emb(abs_pos_input).squeeze()
        pred_wo_summary = content + salience + self.output_bias + rel_pos_embedding + abs_pos_embedding # [32, 30]
        
        pred_summary = pred_wo_summary.clone().detach()
        
        novelty_vector = torch.zeros((batch_size, sentences_count), dtype=torch.float32, device="cuda")

        sliding_summary_first_element = torch.zeros((batch_size, self.hidden_size * 2), dtype=torch.float32, device="cuda")
        sliding_summary = torch.zeros((batch_size, sentences_count, self.hidden_size * 2), dtype=torch.float32, device="cuda")
        sliding_summary[:, 0, :] = sliding_summary_first_element.clone() 

        # for each sentence
        for i in range(1, sentences_count):
            # calc p[i-1]
            weighted_sum = self.novelty_linear_layer(torch.tanh(sliding_summary[:, i - 1, :].clone().squeeze())).unsqueeze(2) # -> [32, 256, 1]
            novelty_vector[:, i - 1] = torch.bmm(outputs[:, i - 1, :].clone().unsqueeze(1),  weighted_sum).squeeze() # [32]
            pred_summary[:, i - 1] = pred_summary[:, i - 1].clone()  - novelty_vector[:, i - 1].clone()  
            
            # calc sliding_summary
            final_s = sliding_summary_first_element.clone().detach()
            for j in range(i):
                final_s += torch.matmul(pred_summary[:, j].clone(), outputs[:, j, :].clone().squeeze()) # [32, 256]
            sliding_summary[:, i, :] = final_s.clone()
                  
        return pred_summary

In [ ]:
model = SentenceTaggerRNN(vocabulary.size())
train_model(model, all_train_data, all_test_data, vocabulary, bpe_processor, device_name="cuda", batch_size=32, lr=0.0001)

Trainable params: 5422118
Epoch = 0, Avg Train Loss = 0.6305, Time = 12.27s
Epoch = 0, Avg Train Loss = 0.2640, Time = 10.95s
Epoch = 0, Avg Train Loss = 0.2223, Time = 10.85s


+ +BERT pretrained embeddings

In [8]:
from transformers import BertModel, BertTokenizer

bert_model_path = 'rubert_cased_L-12_H-768_A-12_pt'
tokenizer = BertTokenizer.from_pretrained(bert_model_path)
model = BertModel.from_pretrained(bert_model_path)
embeddings = model.get_input_embeddings()

In [9]:
import random
import math
import razdel
import torch
import numpy as np
from rouge import Rouge


class BatchIterator():
    def __init__(self, records, batch_size, tokenizer, embeddings, shuffle=True, lower=True, max_sentences=30, max_sentence_length=50, device=torch.device('cpu')):
        self.records = records
        self.num_samples = len(records)
        self.batch_size = batch_size
        self.tokenizer = tokenizer
        self.shuffle = shuffle
        self.batches_count = int(math.ceil(self.num_samples / batch_size))
        self.lower = lower
        self.rouge = Rouge()
        self.embeddings = embeddings
        self.max_sentences = max_sentences
        self.max_sentence_length = max_sentence_length
        self.device = device
        
    def __len__(self):
        return self.batches_count
    
    def __iter__(self):
        indices = np.arange(self.num_samples)
        if self.shuffle:
            np.random.shuffle(indices)

        for start in range(0, self.num_samples, self.batch_size):
            end = min(start + self.batch_size, self.num_samples)
            batch_indices = indices[start:end]
            batch_inputs = []
            batch_outputs = []
            rel_pos_inputs = []
            max_sentence_length = 0
            max_sentences = 0
            batch_records = []
            # свой max_sentences для батча
            for data_ind in batch_indices:
                record = self.records[data_ind]
                batch_records.append(record)
                text = record["text"]
                summary = record["title"]
                summary = summary.lower() if self.lower else summary

                if "sentences" not in record:
                    sentences = [sentence.text.lower() if self.lower else sentence.text for sentence in razdel.sentenize(text)][:self.max_sentences]
                else:
                    sentences = record["sentences"]
                max_sentences = max(len(sentences), max_sentences)

                if "oracle_sentences" not in record:
                    calc_score = lambda x, y: calc_single_score(x, y, self.rouge)
                    sentences_indicies = build_oracle_summary_greedy(text, summary, calc_score=calc_score, lower=self.lower, max_sentences=self.max_sentences)[1]
                else:
                    sentences_indicies = record["oracle_sentences"]
                
                # len(sentences)
                # inputs = [list(map(self.vocabulary.get_index, bpe_tokenize(sentence, self.bpe_processor)[:self.max_sentence_length])) for sentence in sentences]
                tokenized_inputs = [self.tokenizer(sentence) for sentence in sentences]
                inputs = [self.embeddings(torch.LongTensor(sent['input_ids'][1:(self.max_sentence_length + 1)])) for sent in tokenized_inputs]
                max_sentence_length = max(max_sentence_length, max([tokens.shape[0] for tokens in inputs]))
                # len(sentences)
                outputs = [int(i in sentences_indicies) for i in range(len(sentences))]
                # rel_pos
                pos = [i / len(sentences) for i in range(len(sentences))]
                bins = np.array([0.25, 0.5, 0.75, 1.0])
                # possible options array([1, 2, 3, 4, 5])
                rel_pos_inds = np.digitize(np.array([p for p in pos]), bins) + 1
                rel_pos_inputs.append(rel_pos_inds)
                batch_inputs.append(inputs)
                batch_outputs.append(outputs)
            # 768 - BERT embeddings size
            tensor_inputs = torch.zeros((self.batch_size, max_sentences, max_sentence_length, 768), dtype=torch.float32, device=self.device)
            tensor_outputs = torch.zeros((self.batch_size, max_sentences), dtype=torch.float32, device=self.device)
            tensor_abs_pos = torch.zeros((self.batch_size, max_sentences), dtype=torch.long, device=self.device)
            tensor_rel_pos = torch.zeros((self.batch_size, max_sentences), dtype=torch.long, device=self.device)
            for i, inputs in enumerate(batch_inputs):
                for j, sentence_tokens in enumerate(inputs):
                    tensor_inputs[i][j][:len(sentence_tokens)] = torch.FloatTensor(sentence_tokens)
            for i, outputs in enumerate(batch_outputs):
                tensor_outputs[i][:len(outputs)] = torch.LongTensor(outputs)
                # abs pos
                tensor_abs_pos[i] = torch.LongTensor([i for i in range(max_sentences)])
            # rel pos
            for i, rel_pos_input in enumerate(rel_pos_inputs):
                tensor_rel_pos[i][:len(rel_pos_input)] = torch.LongTensor(rel_pos_input)            
            
            yield {
                'inputs': (tensor_inputs, tensor_abs_pos, tensor_rel_pos),
                'outputs': tensor_outputs,
                'records': batch_records
            }

In [10]:
train_iterator = BatchIterator(all_train_data, 4, tokenizer, embeddings, shuffle=True, lower=True, max_sentences=30, max_sentence_length=50)

In [11]:
for batch in train_iterator:
    print(batch)
    break

{'inputs': (tensor([[[[ 4.9386e-02,  3.7106e-02, -2.3729e-02,  ..., -3.7180e-02,
           -4.1990e-02,  1.4780e-02],
          [ 6.2526e-03,  8.8256e-03, -1.9241e-02,  ..., -3.9275e-02,
           -8.4364e-03, -9.6094e-03],
          [ 1.5999e-02,  4.4575e-02, -1.7758e-02,  ...,  4.9740e-03,
           -6.5474e-03, -3.5632e-03],
          ...,
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
            0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
            0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
            0.0000e+00,  0.0000e+00]],

         [[-1.4203e-02,  1.5657e-02, -1.8927e-03,  ..., -3.3525e-02,
           -1.9459e-03, -8.4745e-03],
          [ 2.1707e-02,  2.9045e-02, -1.2116e-02,  ..., -3.9621e-02,
           -4.3472e-02,  5.9852e-03],
          [-6.9191e-02,  4.1258e-02, -5.6599e-02,  ...,  1.1496e-02,
            1.5002e-02, -2.2854e-02],
        

In [12]:
batch['inputs'][0].shape

torch.Size([4, 11, 50, 768])

In [13]:
batch['inputs'][1].shape

torch.Size([4, 11])

In [14]:
batch['inputs'][2].shape

torch.Size([4, 11])

In [15]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

from torch.nn.utils.rnn import pack_padded_sequence as pack
from torch.nn.utils.rnn import pad_packed_sequence as unpack

class SentenceEncoderRNN(nn.Module):
    def __init__(self, input_size, embedding_dim, hidden_size, n_layers=3, dropout=0.3, bidirectional=True):
        super(SentenceEncoderRNN, self).__init__()

        num_directions = 2 if bidirectional else 1
        assert hidden_size % num_directions == 0
        hidden_size = hidden_size // num_directions

        self.embedding_dim = embedding_dim
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.dropout = dropout
        self.bidirectional = bidirectional

        self.embedding_layer = nn.Linear(input_size, embedding_dim)
        self.rnn_layer = nn.LSTM(embedding_dim, hidden_size, n_layers, dropout=dropout, bidirectional=bidirectional, batch_first=True)
        self.dropout_layer = nn.Dropout(dropout)

    def forward(self, inputs, hidden=None):
        embedded = self.embedding_layer(inputs)
        outputs, _ = self.rnn_layer(embedded, hidden)
        sentences_embeddings = torch.mean(outputs, 1)
        return sentences_embeddings

class SentenceTaggerRNN(nn.Module):
    def __init__(self,
                 vocabulary_size,
                 token_embedding_dim=256,
                 sentence_encoder_hidden_size=256,
                 hidden_size=256,
                 bidirectional=True,
                 sentence_encoder_n_layers=2,
                 sentence_encoder_dropout=0.3,
                 sentence_encoder_bidirectional=True,
                 n_layers=1,
                 dropout=0.3):
        super(SentenceTaggerRNN, self).__init__()

        num_directions = 2 if bidirectional else 1
        assert hidden_size % num_directions == 0
        hidden_size = hidden_size // num_directions

        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.dropout = dropout
        self.bidirectional = bidirectional

        self.sentence_encoder = SentenceEncoderRNN(vocabulary_size, token_embedding_dim,
                                                   sentence_encoder_hidden_size, sentence_encoder_n_layers, 
                                                   sentence_encoder_dropout, sentence_encoder_bidirectional)
        self.rnn_layer = nn.LSTM(sentence_encoder_hidden_size, hidden_size, n_layers, dropout=dropout,
                           bidirectional=bidirectional, batch_first=True)
        self.dropout_layer = nn.Dropout(dropout)
        self.content_linear_layer = nn.Linear(hidden_size * 2, 1)
        self.document_linear_layer = nn.Linear(hidden_size * 2, hidden_size * 2)
        self.salience_linear_layer = nn.Linear(hidden_size * 2, hidden_size * 2)
        self.tanh_layer = nn.Tanh()
        
        self.output_bias = nn.Parameter(torch.ones(1))
        self.abs_pos_emb = nn.Embedding(30, 1)
        self.rel_pos_emb = nn.Embedding(6, 1)
        self.novelty_tanh_layer = nn.Tanh()
        self.novelty_linear_layer = nn.Linear(hidden_size * 2, hidden_size * 2)

    def forward(self, inputs, hidden=None):
        embeddings_input = inputs[0]
        abs_pos_input = inputs[1]
        rel_pos_input = inputs[2]
        batch_size = embeddings_input.size(0)
        sentences_count = embeddings_input.size(1)
        tokens_count = embeddings_input.size(2)
        emb_size = embeddings_input.size(3)
        embeddings_input = embeddings_input.reshape(-1, tokens_count, emb_size)
        embedded_sentences = self.sentence_encoder(embeddings_input)
        embedded_sentences = embedded_sentences.reshape(batch_size, sentences_count, -1)
        outputs, _ = self.rnn_layer(embedded_sentences, hidden)
        outputs = self.dropout_layer(outputs) # h [32, 30, 256]
        document_embedding = self.tanh_layer(self.document_linear_layer(torch.mean(outputs, 1))) # [32, 256]
        content = self.content_linear_layer(outputs).squeeze(2)
        # self.salience_linear_layer(document_embedding).unsqueeze(2) [32, 256, 1]
        # [32, 30, 256] * [32, 256, 1] -> [32, 30]
        salience = torch.bmm(outputs, self.salience_linear_layer(document_embedding).unsqueeze(2)).squeeze(2)
        rel_pos_embedding = self.rel_pos_emb(rel_pos_input).squeeze()
        abs_pos_embedding = self.abs_pos_emb(abs_pos_input).squeeze()
        pred_wo_summary = content + salience + self.output_bias + rel_pos_embedding + abs_pos_embedding # [32, 30]
        
        pred_summary = pred_wo_summary.clone().detach()
        
        novelty_vector = torch.zeros((batch_size, sentences_count), dtype=torch.float32, device="cuda")

        sliding_summary_first_element = torch.zeros((batch_size, self.hidden_size * 2), dtype=torch.float32, device="cuda")
        sliding_summary = torch.zeros((batch_size, sentences_count, self.hidden_size * 2), dtype=torch.float32, device="cuda")
        sliding_summary[:, 0, :] = sliding_summary_first_element.clone() 

        # for each sentence
        for i in range(1, sentences_count):
            # calc p[i-1]
            weighted_sum = self.novelty_linear_layer(torch.tanh(sliding_summary[:, i - 1, :].clone().squeeze())).unsqueeze(2) # -> [32, 256, 1]
            novelty_vector[:, i - 1] = torch.bmm(outputs[:, i - 1, :].clone().unsqueeze(1),  weighted_sum).squeeze() # [32]
            pred_summary[:, i - 1] = pred_summary[:, i - 1].clone()  - novelty_vector[:, i - 1].clone()  
            
            # calc sliding_summary
            final_s = sliding_summary_first_element.clone().detach()
            for j in range(i):
                final_s += torch.matmul(pred_summary[:, j].clone(), outputs[:, j, :].clone().squeeze()) # [32, 256]
            sliding_summary[:, i, :] = final_s.clone()
                  
        return pred_summary

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
import time

def train_model(model, train_records, val_records, tokenizer, embeddings, batch_size=32,
                epochs_count=10, loss_every_nsteps=16, lr=0.001, device_name="cuda"):
    params_count = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print("Trainable params: {}".format(params_count))
    device = torch.device(device_name)
    model = model.to(device)
    total_loss = 0
    start_time = time.time()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    loss_function = nn.BCEWithLogitsLoss().to(device)
    for epoch in range(epochs_count):
        for step, batch in enumerate(BatchIterator(train_records, batch_size, tokenizer, embeddings, device=device)):
            model.train()
            logits = model(batch["inputs"]) # Прямой проход
            loss = loss_function(logits, batch["outputs"]) # Подсчёт ошибки
            loss.backward() # Подсчёт градиентов dL/dw
            optimizer.step() # Градиентный спуск или его модификации (в данном случае Adam)
            optimizer.zero_grad() # Зануление градиентов, чтобы их спокойно менять на следующей итерации
            total_loss += loss.item()
            if step % loss_every_nsteps == 0 and step != 0:
                val_total_loss = 0
                val_batch_count = 0
                model.eval()
#                 for _, val_batch in enumerate(BatchIterator(val_records, vocabulary, batch_size, bpe_processor, device=device)):
#                     logits = model(val_batch["inputs"]) # Прямой проход
#                     val_total_loss += loss_function(logits, val_batch["outputs"]) # Подсчёт ошибки
#                     val_batch_count += 1
#                 avg_val_loss = val_total_loss/val_batch_count
#                 print("Epoch = {}, Avg Train Loss = {:.4f}, Avg val loss = {:.4f}, Time = {:.2f}s".format(epoch, total_loss / loss_every_nsteps, avg_val_loss, time.time() - start_time))
                print("Epoch = {}, Avg Train Loss = {:.4f}, Time = {:.2f}s".format(epoch, total_loss / loss_every_nsteps, time.time() - start_time))
                total_loss = 0
                start_time = time.time()
        total_loss = 0
        start_time = time.time()

In [ ]:
model = SentenceTaggerRNN(768)
train_model(model, all_train_data, all_test_data, tokenizer, embeddings, device_name="cuda", batch_size=32, lr=0.0001)

/home/ksenya/anaconda3/envs/nlp_project/lib/python3.6/site-packages/torch/nn/modules/rnn.py:61: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Trainable params: 1580326
Epoch = 0, Avg Train Loss = 1.2350, Time = 905.25s
Epoch = 0, Avg Train Loss = 0.9494, Time = 776.92s
Epoch = 0, Avg Train Loss = 0.9191, Time = 767.12s
Epoch = 0, Avg Train Loss = 0.8978, Time = 752.85s
Epoch = 0, Avg Train Loss = 0.8994, Time = 777.80s
Epoch = 0, Avg Train Loss = 0.8790, Time = 749.86s
Epoch = 0, Avg Train Loss = 0.9475, Time = 796.74s
Epoch = 0, Avg Train Loss = 0.8996, Time = 783.21s
Epoch = 0, Avg Train Loss = 0.8862, Time = 801.85s
Epoch = 0, Avg Train Loss = 0.8643, Time = 756.11s
Epoch = 0, Avg Train Loss = 0.8750, Time = 739.63s
Epoch = 0, Avg Train Loss = 0.8646, Time = 775.75s
Epoch = 0, Avg Train Loss = 0.8181, Time = 765.45s
Epoch = 0, Avg Train Loss = 0.7215, Time = 736.95s
Epoch = 0, Avg Train Loss = 0.6706, Time = 742.47s
Epoch = 0, Avg Train Loss = 0.2816, Time = 764.29s
Epoch = 0, Avg Train Loss = 0.2229, Time = 783.32s
Epoch = 0, Avg Train Loss = 0.1941, Time = 791.25s
Epoch = 0, Avg Train Loss = 0.1908, Time = 751.35s
Epoch

Уменьшили количество параметров за счет предобученных эмбеддингов, однако runtime значительно увеличился